***
# <font color=indianred size=10>INTRODUÇÃO</font>

***
- Arquivo	: notebook.ipynb
- Título	: Exercício 4 - Disciplina de Redes Neurais Artificiais (DELT/UFMG)
- Autor	    : Gustavo Augusto Ortiz de Oliveira (gstvortiz@hotmail.com) <br> <br> <br>
- Descrição: 
A prática desta semana introduz Redes Neurais do tipo RBF (Radial Basis Functions), que são caracterizadas pela utilização de funções radiais nos neurônios de sua única camada intermediária. Similar às estruturas estudadas anteriormente, essas respostas são combinadas linearmente para a obtenção de saída da rede. De maneira geral, uma rede RBF pode ser descrita conforme equação:
$$  f(x,\theta) = \sum_{i=1}^{n}w_{i}h_{i}(x, z_{i})+w_{0} $$

Nesse contexto, temos que:
1. **$w_i$** é o vetor de parâmetros do neurônio de saída.
2. **$z_i$** é o vetor que contém todos os parâmetros do neurônio $i$ da camada intermediária.
3. **$\theta$** é o vetor que contém a concatenação de todos os parâmetros.



## [1]. BIBILIOTECAS

In [1]:
import os
import shutil
import datetime
from PIL import Image
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import pi as pi
from sklearn.cluster import KMeans as SKMeans
from sklearn.model_selection import train_test_split

np.set_printoptions(suppress=True, precision=3)
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300})
sns.set_style('ticks')

***
# <font color=royalblue size=10>FUNÇÕES</font>

***


## [1] SEPARAR DADOS

In [2]:
def SepararDados(dir, col = 'labels', sample = None):
    dados = pd.read_csv(dir).sample(sample) if sample else pd.read_csv(dir)
    X = dados.drop(col, axis = 1).values
    labels = dados[col].values
    return X, labels

## [2] KMEANS

In [3]:
def KMeans(X, p, n_init = 10, func = np.mean):
    kmeans = SKMeans(n_clusters=p, n_init=n_init).fit(X)
    labels = kmeans.labels_ 
    centros = kmeans.cluster_centers_
    raios = list()
    for cluster_id in range(p):
        cluster_points = X[labels == cluster_id]
        centro = centros[cluster_id]
        distancias_euclidianas = np.linalg.norm(cluster_points - centro, axis=1)
        raios.append(func(distancias_euclidianas))
    return centros, raios, labels

## [3] CLASSE RBF

In [4]:
def Normal(c, r):
    return lambda x: np.exp(-((x - np.array(c))**2).sum() / (2 * r**2))

class NormalRBF:
    def __init__(self, centros, raios):
        self.centros = centros
        self.raios = raios
        self.p = len(raios)
        self.w = None

    def HiddenLayer(self, X):
        N = X.shape[0]
        H = np.ones((N, self.p + 1))
        for i in range(N):
            for j in range(self.p):
                H[i, j] = Normal(self.centros[j], self.raios[j])(X[i])
        return H

    def fit(self, X, labels):
        self.w = np.linalg.pinv(self.HiddenLayer(X)).dot(labels)
    
    def predict(self, X):
        return self.HiddenLayer(X).dot(self.w)

## [4] CLASSE PARA VISUALIZAÇÃO

In [5]:
class Fig():
    def __init__(self, rows = 2, cols = 2, fs = (10, 7), lim = 3):
        self.fig, self.axs  = plt.subplots(rows, cols, figsize=fs)
        for ax in self.axs.ravel():
            ax.set_xlim((-lim, lim))
            ax.set_ylim((-lim, lim))

    def GridHelper(self, ax):
        eixo_x = np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], 200)
        eixo_y = np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 200)
        xx, yy = np.meshgrid(eixo_x, eixo_y)
        pontos = np.column_stack((xx.ravel(), yy.ravel()))
        return xx, yy, pontos

    def Grid(self, n, Neuronio, alpha = 0.7, palette = 'plasma'):
        ax = self.axs.ravel()[n]
        xx, yy, pontos = self.GridHelper(ax)
        labels_pred = Neuronio.predict(pontos).reshape(xx.shape).round(2)
        ax.contourf(xx, yy, labels_pred, alpha = alpha, cmap = palette)
    
    def Plot(self, n, x0, x1, labels, plot = sns.scatterplot, palette = 'plasma'):
        ax = self.axs.ravel()[n]
        plot(x = x0, y = x1, hue = labels, ax = ax, palette = palette)
        ax.get_legend().remove()

## [5] CLASSE PARA GIF

In [6]:
class GIF():
    def __init__(self, folder = 'data/img'):
        self.folder = folder

    def Preparar(self):
        if os.path.exists(self.folder):
            shutil.rmtree(self.folder)
        os.mkdir(self.folder)
    
    def FileNames(self):
        arquivos_info = []
        for nome_imagem in os.listdir(self.folder):
            caminho_completo = os.path.join(self.folder, nome_imagem)
            data_criacao = datetime.datetime.fromtimestamp(os.path.getctime(caminho_completo))
            arquivos_info.append((caminho_completo, data_criacao))
        arquivos_info.sort(key=lambda x: x[1])
        return np.array(arquivos_info)[:, 0]
    
    def Make(self, path = 'data'):
        imagens = []
        for caminho in self.FileNames():
            imagem = Image.open(caminho)
            imagens.append(imagem)
        imagens[0].save(f'{path}/gif.gif', save_all=True, append_images=imagens[1:], duration=400, loop=0)

***
# <font color=forestgreen size=10>EXERCÍCIO PROPOSTO</font>
*** 
<br>
<div align = 'center'> <img src = 'data/Enunciado.png'> </div> <br>

***

## [1] RESOLUÇÃO COMENTADA

In [7]:
# Configurações Preliminares
GIF().Preparar()
acuracias = dict()

for sample in range(100, 2001, 10):
    # [1] Preparando Dados
    X, labels = SepararDados('data/xor.csv', sample = sample)
    X_train, X_test, labels_train, labels_test = train_test_split(X, labels, train_size=0.9, stratify=labels)
    centros, raios = KMeans(X_train, p = 4)[:2]

    # [2] Treinando a Rede
    Rede = NormalRBF(centros, raios)
    Rede.fit(X_train, labels_train)

    ## [3] Realizando Previsões
    labels_pred = Rede.predict(X_test)

    ## [4] Calculando Acurácia do Modelo
    bool_vector = (np.round(labels_pred) == labels_test)
    acuracias[sample] = 100 * sum(bool_vector)/len(bool_vector)

    # [5] Armazenando Visualização de Resultados
    fig = Fig(1, 2, (10, 4), 4)
    fig.fig.suptitle(f'Radial Basis Function Neural Network: Problema XOR ({sample} Amostras)', fontsize = 14, y = 1.08)
    fig.axs[0].set_title(f'Dados de Treino ({len(labels_train)})')
    fig.Plot(0, X_train[:, 0], X_train[:, 1], labels_train)
    fig.axs[1].set_title(f'Dados de Teste ({len(labels_test)})')
    fig.Grid(1, Rede, 0.5)
    fig.Plot(1, X_test[:, 0], X_test[:, 1], labels_test)
    ax = fig.fig.add_axes([0.15, -0.5, 0.7, 0.4])
    ax.set_title('Precisão do Modelo por Quantidade de Amostras')
    sns.lineplot(x = acuracias.keys(), y = acuracias.values())
    ax.set_xlabel('Quantidade de Amostras')
    ax.set_ylabel('Acurácia (%)')
    ax.set_ylim(90, 101)
    plt.savefig(f'data/img/{sample}.png', bbox_inches = 'tight')
    plt.close()

# Criando GIF com Resultados
GIF().Make()

## [2] VISUALIZANDO RESULTADOS
<div> <img src = "data/gif.gif"> </div>

$$ X= \begin{bmatrix}
x_{11} & x_{12} & \ldots & x_{1n} \\
x_{21} & x_{22} & \ldots & x_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
x_{N1} & x_{N2} & \ldots & x_{Nn} \\
\end{bmatrix}_ {Nxn}  $$

$$ c =
\begin{bmatrix}
c_{11} & c_{12} & \ldots & c_{1p} \\
c_{21} & c_{22} & \ldots & c_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
c_{n1} & c_{n2} & \ldots & c_{np} \\
\end{bmatrix}_ {nxp} $$

$$ r =
\begin{bmatrix}
r_{1} &
r_{2} &
\ldots &
r_{n}
\end{bmatrix} $$